# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB
df = pd.read_csv('TrailMakeB_with_MethylomeDetails.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB
(216, 61926)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Trail_Making_B_Category,cg02315971,cg24133836,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.030109,0.016106,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.046350,0.904242,0.021033,0.813738
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.028697,0.017000,...,0.109870,0.056582,0.064137,0.053802,0.083556,0.042638,0.041733,0.896521,0.033285,0.753096
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.025308,0.017806,...,0.179598,0.037220,0.050652,0.027469,0.056702,0.032307,0.024831,0.924415,0.016215,0.900643
3,200989060105_R03C01,3003,2,1,2,56.7,3,0,0.023382,0.019290,...,0.114735,0.031287,0.041654,0.033260,0.040730,0.032994,0.021361,0.840979,0.034946,0.858052
4,200989060135_R07C01,3004,1,2,0,59.4,3,1,0.025681,0.020248,...,0.136500,0.038891,0.039595,0.026053,0.051206,0.029660,0.027424,0.868682,0.020491,0.888800


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'Trail_Making_B_Category','Age','AgeCate'], axis=1)
print(df1.shape)
df1.head(2)

(216, 61919)


,APPRDX,cg02315971,cg24133836,cg03286969,cg11738782,cg25741731,cg24629744,cg23836745,cg11685489,cg06081147,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,2,0.030109,0.016106,0.019371,0.884151,0.026326,0.868373,0.905809,0.825562,0.691096,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.046350,0.904242,0.021033,0.813738
1,1,0.028697,0.017000,0.018631,0.871526,0.026148,0.866396,0.922285,0.881489,0.647436,...,0.109870,0.056582,0.064137,0.053802,0.083556,0.042638,0.041733,0.896521,0.033285,0.753096


## RanFor 100 times, CpG only 

In [5]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [6]:
dframe1.to_csv ("TrailMakeB_AllCpG_Accuracy_100RF.csv", index=False)

In [7]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.475000,0.083749
1,0.516667,0.116667


## RanFor 1000 times. Samples = 50

In [8]:
df2 = df1.drop (['APPRDX'], axis=1)

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [10]:
print(dframe1.head(3))
dframe1.to_csv ("TrailMakeB_Rand50CpG_100RF.csv")

   Accuracy   Std_Dev
0  0.608333  0.075000
1  0.466667  0.119024
2  0.491667  0.168531


## RanFor 1000 times. Samples = 6

In [11]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(6, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [12]:
print(dframe1.head(3))
dframe1.to_csv ("TrailMakeB_Rand6CpG_100RF.csv")

   Accuracy   Std_Dev
0  0.608333  0.075000
1  0.466667  0.119024
2  0.491667  0.168531
